In [33]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='masked_df2.csv')

Using custom data configuration default-fb8052464dae2b4c


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 90.61it/s]


Dataset csv downloaded and prepared to /Users/samet/.cache/huggingface/datasets/csv/default-fb8052464dae2b4c/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 339.32it/s]


In [36]:
dataset = dataset.remove_columns('id')

In [37]:
dataset = dataset['train'].train_test_split(test_size=0.1)

In [38]:
dataset['train'].features

{'labels': Value(dtype='string', id=None),
 'snippet': Value(dtype='string', id=None)}

In [41]:
def tokenize_function(example):
    return tokenizer(example['snippet'], example['labels'], padding='max_length', truncation=True, max_length=512)

In [42]:
tokenized_dataset = dataset.map(tokenize_function)

100%|██████████| 2213/2213 [00:01<00:00, 1337.21ex/s]


In [43]:
tokenized_dataset.column_names

{'train': ['labels', 'snippet', 'input_ids', 'attention_mask'],
 'test': ['labels', 'snippet', 'input_ids', 'attention_mask']}

In [17]:
tokenized_dataset = tokenized_dataset.remove_columns(['snippet'])
tokenized_dataset = tokenized_dataset.with_format('torch')

In [18]:
tokenized_dataset['train']

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 22124
})

In [19]:
small_dataset = tokenized_dataset['train'].select(range(100))

In [20]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

In [24]:
from transformers import AutoModel, Trainer
model_id = "huggingface/CodeBERTa-small-v1"
model = AutoModel.from_pretrained(model_id)

Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['test'],
    tokenizer = tokenizer,
)

In [26]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaModel.forward` and have been ignored: labels. If labels are not expected by `RobertaModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22124
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8298
  0%|          | 0/8298 [01:15<?, ?it/s]


KeyError: 'loss'